In [ ]:
from google.colab import drive
drive.mount('/gdrive',force_remount=True)

Mounted at /gdrive


In [ ]:
!unzip '/gdrive/My Drive/FMD12k.zip' -d '/temp2/'

In [ ]:
import numpy as np
import keras
import keras.backend as k
import matplotlib.pyplot as plt
from keras.layers import Conv2D,MaxPooling2D,SpatialDropout2D,Flatten,Dropout,Dense
from keras.models import Sequential,load_model
from keras.optimizers import Adam
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
import cv2
import datetime
import os


In [ ]:
model=Sequential()
model.add(Conv2D(32,(3,3),activation='relu',input_shape=(150,150,3)))
model.add(MaxPooling2D() )
model.add(Conv2D(32,(3,3),activation='relu'))
model.add(MaxPooling2D() )
model.add(Conv2D(32,(3,3),activation='relu'))
model.add(MaxPooling2D() )
model.add(Flatten())
model.add(Dense(100,activation='relu'))
model.add(Dense(1,activation='sigmoid'))
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 148, 148, 32)      896       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 74, 74, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 72, 72, 32)        9248      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 36, 36, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 34, 34, 32)        9248      
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 17, 17, 32)        0         
_________________________________________________________________
flatten (Flatten)            (None, 9248)              0

In [ ]:

model.compile(optimizer = Adam(lr = 0.001), loss = 'binary_crossentropy', metrics = ['accuracy'])

In [ ]:
train_datagen = ImageDataGenerator(
                  rescale = 1./255,
                  shear_range = 0.2,
                  zoom_range = 0.2,
                  horizontal_flip = True)

test_datagen = ImageDataGenerator(rescale = 1./255, horizontal_flip = True)

training_set = train_datagen.flow_from_directory(
              '/temp2/Face Mask Dataset/Train',
              target_size = (150, 150),
              batch_size = 16 ,
              class_mode = 'binary')

test_set = test_datagen.flow_from_directory(
              '/temp2/Face Mask Dataset/Test',
              target_size = (150, 150),
              batch_size = 16,
              class_mode = 'binary')


Found 10000 images belonging to 2 classes.
Found 992 images belonging to 2 classes.


In [ ]:
filepath="/gdrive/My Drive/weights/zweights-{val_loss:.4f}.h5"
checkpoint = keras.callbacks.ModelCheckpoint(filepath, 
                                              monitor = 'val_loss', 
                                              verbose = 1, 
                                              save_best_only = True, 
                                              save_weights_only = True, 
                                              mode = 'auto', 
                                              period = 1)
callbacks_list = [checkpoint]

In [ ]:
batch_size=16

In [ ]:
history = model.fit_generator(training_set, 
                    epochs = 50,  
                    verbose = 1,
                    validation_data = test_set,  
                    callbacks = callbacks_list)
print(history.history.keys())

plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()